In [ ]:
from ipynb.fs.full.logistic_regression_functions import *

LABELS_PATH = "data/labels_kmeans"
DATA_FEATURES_DF = 'data/df_from_mean_fasttext_15_5_400.pkl'
KMEANS_CLUSTER_IDS = 'data/cluster_ids.pkl'

CLASSIFIER_PATH = "models/model.pkl"
TRANSFORMER_PATH = "models/transformer.pkl"

### Load Corpus 

In [ ]:
data, original_data = load_corpus()

### Get features of corpus and labels from kmeans

In [ ]:
# Features to train the classifier.
data_feature = pd.read_pickle(DATA_FEATURES_DF)
# Ids of the cluster for each sentence.
cluster_ids = pd.read_pickle(KMEANS_CLUSTER_IDS)
# Classes for the classifier.
with open(LABELS_PATH, 'r') as file:
    labels = file.read()
labels = labels.split('\n)

### Prepare data to train the classifier

In [ ]:
# join vectors and labels
text_label = [[data_feature.loc[ind].values, cluster_ids.loc[ind,0]] for ind in range(0,len(data_feature))]

# create DataFrame
df = pd.DataFrame(text_label, columns = ['text','label'])

data_as_str = [' '.join(x) for x in data]

text_as_str_label = [[data_as_str[ind], cluster_ids.loc[ind,0]] for ind in range(0, len(data_feature))]
df_with_str = pd.DataFrame(text_as_str_label, columns = ['text','label'])

### Train model with function 1

In [ ]:
field='text'
top_k=3

model,accuracy,mrr_at_k=train_model(df)
print("\nAccuracy={0}; MRR={1}".format(accuracy,mrr_at_k))

### Training model with function 2

In [ ]:
field='text'
top_k=3
feature_rep = 'tf-idf' #binary, counts, tf-idf 

model, feature_transformer, accuracy, mrr = train_model_with_transformer(df_with_str, feature_rep=feature_rep)
print("\nAccuracy={0}; MRR={1}".format(accuracy,mrr_at_k))

test_features = feature_transformer.transform(['hola queres ir a jugar al tenis'])
labels_ind = get_top_k_predictions(model, test_features, 2)
print('Los labels para este ejemplo son:', [labels[x] for x in labels_ind[0]])

### Save model and transformer

In [ ]:
# we need to save both the transformer and model 
# transformer to encode/vectorize per our settings
# model to predict
pickle.dump(model,open(CLASSIFIER_PATH, 'wb'))
pickle.dump(feature_transformer,open(TRANSFORMER_PATH,'wb'))